In [55]:
import torch
from torch.nn import functional
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [56]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [57]:
tokenizer = AutoTokenizer.from_pretrained('./saved_model/gpt_sentiment_tokenizer')

In [58]:
model = AutoModelForSequenceClassification.from_pretrained('./saved_model/gpt_sent_trainer',num_labels=3)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at openai-community/gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [59]:
inputs = tokenizer("I love her",padding="max_length",truncation=True,max_length=128,return_tensors='pt')

In [60]:
print(inputs)

{'input_ids': tensor([[   40,  1842,   607, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 5

In [61]:
outputs = model(**inputs)

In [62]:
torch.argmax(outputs.logits,dim=1)

tensor([2])

In [63]:
label_map = {0: "Negative", 1: "Neutral", 2: "Positive"}

In [64]:
label_map[torch.argmax(outputs.logits,dim=1).item()]

'Positive'

In [65]:
# Evaluate Model with Example Texts
def evaluate_example_texts(texts, true_labels):
    model.eval()
    model.to(device)
    correct, total = 0, 0
    label_map = {0: "Negative", 1: "Neutral", 2: "Positive"}

    with torch.no_grad():
        for text, true_label in zip(texts, true_labels):
            inputs = tokenizer(text, truncation=True, return_tensors='pt')
            # input_ids, attention_mask = inputs["input_ids"].to(device), inputs["attention_mask"].to(device)
            outputs = model(**inputs)

            probs = functional.softmax(outputs.logits,dim=-1)
            pred_label =  torch.argmax(probs,dim=1).item() # Placeholder logic for sentiment extraction from Llama output
            confidence = probs[0,pred_label].item()
            print(f"Text: {text}\nPredicted Sentiment: {label_map[pred_label]} (Actual: {label_map[true_label]}), (Confidence = {confidence}) \n")
            if pred_label == true_label:
                correct += 1
            total += 1

    accuracy = correct / total
    print(f"Evaluation Accuracy: {accuracy:.4f}")

In [66]:
# Test Example Texts
test_texts = [
    "I love this product! It works perfectly.",
    "This is the worst service I've ever received.",
    'I Love her'
]
test_labels = [2, 0, 2]  # Expected sentiments: Positive, Negative, Neutral
evaluate_example_texts(test_texts, test_labels)

Text: I love this product! It works perfectly.
Predicted Sentiment: Positive (Actual: Positive), (Confidence = 0.8515491485595703) 

Text: This is the worst service I've ever received.
Predicted Sentiment: Negative (Actual: Negative), (Confidence = 0.8594591021537781) 

Text: I Love her
Predicted Sentiment: Positive (Actual: Positive), (Confidence = 0.8062491416931152) 

Evaluation Accuracy: 1.0000
